In [23]:
import requests
from bs4 import BeautifulSoup
import time
import re

def scrape_google_repos_complete():
    url = "https://github.com/orgs/google/repositories?q=&type=all&language=&sort=stargazers"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    }

    print("スクレイピングを開始します...")
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"アクセスエラー: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # ★実績のある「h3タグ」から攻める（これが一番確実）
    all_h3 = soup.find_all('h3')
    
    data_list = []
    count = 0

    print("-" * 60)
    print(f"{'No.':<4} | {'リポジトリ名':<30} | {'言語':<10} | {'スター数'}")
    print("-" * 60)

    for h3 in all_h3:
        # h3の中のリンクを取得
        a_tag = h3.find('a')
        if not a_tag: continue
        
        # Googleのリポジトリじゃないリンク（ナビゲーションなど）は無視
        href = a_tag.get('href', '')
        if not href.strip().startswith('/google/'): continue
            
        name = a_tag.text.strip()
        
        # 親要素（行全体）を取得
        parent = h3.find_parent('li')
        
        # もしliが親じゃなかった場合（divなどの場合）、もっと上の階層を探す
        if not parent:
            parent = h3.find_parent('div', class_='Box-row')
        if not parent: # それでもなければh3の親の親を使う
            parent = h3.parent.parent
        
        # --- 言語取得ロジック（あなたの発見を取り入れました！） ---
        language = "Unknown"
        
        if parent:
            # 1. まず「色のついた丸ポチ」があるか探す（これが一番正確）
            color_dot = parent.find('span', style=lambda s: s and 'background-color' in s)
            
            if color_dot:
                # 丸ポチの親のテキストを取る
                language = color_dot.parent.text.strip()
            else:
                # 2. 丸ポチがない場合、あなたが発見したクラス名を探す
                # "ReposListItem-module__Text" を含むタグを全部探す
                target_spans = parent.find_all('span', class_=lambda c: c and 'ReposListItem-module__Text' in c)
                
                for span in target_spans:
                    text = span.text.strip()
                    # 「Updated」や「License」ではなく、かつ数字だけでもないものを言語とみなす
                    if "Updated" not in text and "License" not in text and not text.isdigit() and len(text) < 20:
                        language = text
                        break

        # --- スター数取得 ---
        stars = "0"
        # "Stargazers" または "stars" を含むリンクを探す
        star_tag = parent.find('a', href=lambda x: x and 'stargazers' in x)
        if star_tag:
            stars = star_tag.text.strip().replace(',', '')
        else:
            # 新デザイン対応：aria-labelにstarが含まれるものを探す
            star_span = parent.find('span', id=lambda x: x and 'star' in x)
            if star_span:
                stars = star_span.text.strip().replace(',', '')

        # データの整形
        language = language.replace('\n', '').strip()
        
        # 表示
        print(f"{count+1:<4} | {name:<30} | {language:<10} | {stars}")
        data_list.append((name, language, stars))
        
        time.sleep(1)
        count += 1
        if count >= 20: break

    print("-" * 60)
    print(f"スクレイピング完了: {len(data_list)} 件取得しました。")
    return data_list

# 実行
scraped_data = scrape_google_repos_complete()

スクレイピングを開始します...
------------------------------------------------------------
No.  | リポジトリ名                         | 言語         | スター数
------------------------------------------------------------
1    | material-design-icons          | Unknown    | 53k
2    | guava                          | Java       | 51k
3    | zx                             | JavaScript | 45k
4    | styleguide                     | HTML       | 39k
5    | leveldb                        | C++        | 38k
6    | googletest                     | C++        | 37k
7    | comprehensive-rust             | Rust       | 32k
8    | material-design-lite           | HTML       | 32k
9    | python-fire                    | Python     | 28k
10   | flatbuffers                    | C++        | 25k
11   | gson                           | Java       | 24k
12   | ExoPlayer                      | Java       | 22k
13   | iosched                        | Kotlin     | 22k
14   | eng-practices                  | Unknown    | 20k
15   

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_google_repos_all():
    # ベースとなるURL（ページ番号以外）
    base_url = "https://github.com/orgs/google/repositories?q=&type=all&language=&sort=stargazers"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    }

    all_data = []
    page = 1
    
    max_pages = 100

    print(f"スクレイピングを開始します（最大 {max_pages} ページまで取得）...")
    print("-" * 60)
    print(f"{'No.':<4} | {'リポジトリ名':<30} | {'言語':<10} | {'スター数'}")
    print("-" * 60)
    
    total_count = 0

    # ページをめくるループ
    while page <= max_pages:
        print(f"\n--- {page}ページ目を読み込み中 ---")
        
        # URLに &page=ページ番号 をつける
        target_url = f"{base_url}&page={page}"
        
        try:
            response = requests.get(target_url, headers=headers, timeout=10)
            response.raise_for_status()
        except Exception as e:
            print(f"ページ取得エラー: {e}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # リポジトリ名（h3）を取得
        all_h3 = soup.find_all('h3')
        
        # もしこのページにリポジトリがなければ終了（最後のページ）
        if not all_h3:
            print("これ以上リポジトリがありません。終了します。")
            break
            
        items_on_page = 0

        for h3 in all_h3:
            a_tag = h3.find('a')
            if not a_tag: continue
            href = a_tag.get('href', '')
            if not href.strip().startswith('/google/'): continue
                
            name = a_tag.text.strip()
            
            # 親要素を探す
            parent = h3.find_parent('li')
            if not parent: parent = h3.find_parent('div', class_='Box-row')
            if not parent: parent = h3.parent.parent
            
            language = "Unknown"
            stars = "0"
            
            if parent:
                # 言語取得
                color_dot = parent.find('span', style=lambda s: s and 'background-color' in s)
                if color_dot:
                    language = color_dot.parent.text.strip()
                else:
                    target_spans = parent.find_all('span', class_=lambda c: c and 'ReposListItem-module__Text' in c)
                    for span in target_spans:
                        text = span.text.strip()
                        if "Updated" not in text and "License" not in text and not text.isdigit() and len(text) < 20:
                            language = text
                            break

                # スター数取得
                star_tag = parent.find('a', href=lambda x: x and 'stargazers' in x)
                if star_tag:
                    stars = star_tag.text.strip().replace(',', '')
                else:
                    star_span = parent.find('span', id=lambda x: x and 'star' in x)
                    if star_span:
                        stars = star_span.text.strip().replace(',', '')

            language = language.replace('\n', '').strip()
            
            # 表示とリスト追加
            total_count += 1
            print(f"{total_count:<4} | {name:<30} | {language:<10} | {stars}")
            all_data.append((name, language, stars))
            
            items_on_page += 1
            
            # ★ここ大事！1件ごとに1秒待つ（課題ルール）
            time.sleep(1)
        
        # もしこのページで何も取れなかったら終了
        if items_on_page == 0:
            print("データが見つかりませんでした。終了します。")
            break
            
        # 次のページへ
        page += 1
        # ページ遷移の間にも少し休憩（サーバー負荷軽減）
        time.sleep(2)

    print("-" * 60)
    print(f"全処理完了: 合計 {len(all_data)} 件取得しました。")
    return all_data

# 実行
scraped_data = scrape_google_repos_all()